## Анализ характеристик песен из плейлистов Spotify для обучения
## (Analysis of the song features for the Spotify playlist for studying)

Цель проекта: изучение характеристик песен из плейлистов для обучения

Задачи:

1) Создание базы данных с треками из специализированных плейлистов Spotify, которую можно будет в дальнейшем ипользовать для рекомендаций пользователям

2) Изучение основных характеристик песен из плейлистов

3) Кластеризация песен из плейлистов на основании характеристик. Есть ли различия между песнями в плейлистах для обучения?

4) Сравнение характеристик песен из плейлистов для учебы и песен из общей базы Spotify 

## Настройка подключения к Spotify API

Для использования Spotify API нужно зарегистрировать приложение на сайте spotify и получить токен. Буду работать с библиотекой spotipy, помогающей отправлять запросы к API. Подробнее про API можно почитать в документации: https://developer.spotify.com/discover/

In [ ]:
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 

cid = #user_id
secret = #secret_id

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

## Получение информацию о песнях для трех плейлистов для учебы

Были выбраны плейлисты: "Погружение в учебу", "Instrumental Study", "Intense Studuing"

In [ ]:
playlist_ids=['37i9dQZF1DX8NTLI2TtZa6','37i9dQZF1DX9sIqqvKsjG8','37i9dQZF1DWZeKCadgRdKQ','0vvXsWCC9xrXsKd4FyS8kM']
tracklist=[]
len(tracklist)

In [ ]:
results=sp.playlist_items(playlist_ids[3])
len(results['items'])

In [ ]:
for playlist in playlist_ids:
    print(playlist)

Как можно увидеть, API выдает в одном результате максимум 100 элементов. Поэтому нужно итеративно с каждым разом добавлять по 100 треков для каждого плейлиста. Почему-то у меня не вышло одним лупом спарсить информацию о песнях сразу для трех плейлистов, поэтому буду делать по очереди.

In [ ]:
#Выгружаю информацию для первого плейлиста
tracklist=[]
i=0
while True:
    if len(results['items']) != 0:
        results=sp.playlist_items(playlist_ids[0], offset=i)
        for item in results['items']:
            tracklist.append(item['track']['id'])
        i+=100
    else:
        break
len(tracklist)

In [ ]:
results=sp.playlist_items(playlist_ids[3])
results['items'][0]['track']['name']

In [ ]:
#Выгружаю информацию для второго плейлиста, добавляю в общий список
i=0
while True:
    if len(results['items']) != 0:
        results=sp.playlist_items(playlist_ids[1], offset=i)
        #track_name = results['items'][0]['track']['name']
        #artist_name = results['items'][0]['track']['artists'][0]['name']
        for item in results['items']:
            tracklist.append(item['track']['id'])
        i+=100
    else:
        break
len(tracklist)

In [ ]:
results=sp.playlist_items(playlist_ids[2])
results['items'][0]['track']['name']

In [ ]:
#Выгружаю информацию для третьего плейлиста, добавляю в общий список
i=0
while True:
    if len(results['items']) != 0:
        results=sp.playlist_items(playlist_ids[2], offset=i)
        #track_name = results['items'][0]['track']['name']
        #artist_name = results['items'][0]['track']['artists'][0]['name']
        for item in results['items']:
            tracklist.append(item['track']['id'])
        i+=100
    else:
        break
len(tracklist)

In [ ]:
results=sp.playlist_items(playlist_ids[3])
results['items'][0]['track']['name']

In [ ]:
#Выгружаю информацию для четвертого плейлиста, добавляю в общий список
i=0
while True:
    if len(results['items']) != 0:
        results=sp.playlist_items(playlist_ids[3], offset=i)
        #track_name = results['items'][0]['track']['name']
        #artist_name = results['items'][0]['track']['artists'][0]['name']
        for item in results['items']:
            tracklist.append(item['track']['id'])
        i+=100
    else:
        break
len(tracklist)

In [ ]:
sp.audio_features(tracklist[1012])

In [ ]:
tracklist.pop(1010)

## Создание базы данных с информацией о песнях из плейлистов для учебы

In [ ]:
#В связи с ограничением на максимум 100 объектов в одном запросе, нужно изощряться.
#Создаю цикл специально для полученного числа треков.

import pandas as pd

for i in range(0,11):
    if i == 0:
        df=pd.DataFrame(sp.audio_features(tracks=tracklist[:100]))
    elif i in range (1,10):
        num=i*100
        df1=pd.DataFrame(sp.audio_features(tracks=tracklist[num:num+100]))
        df=df.append(df1)
df = df.reset_index(drop=True) #обновляю индексы, т.к. они сбились при объединении баз данных
len(df)

In [ ]:
df.head()

Оставляю только характеристики треков. Темп и тональность не включаю, т.к. проблематично анализировать эти значения (нужно переводить в другие категории + более обширный музыкальный бэкграунд, чтобы понять, о чем это)

In [ ]:
df=df[['id','acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','valence']]
df.describe()

## Анализ характеристик песен

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 10))
sns.heatmap(df[1:].corr(), annot=True)

Самая высокая корреляция наблюдается между энергичностю и громкостью: 0,77. Также наблюдаются высокие по модулю, но отрицательные корреляции между акустичностью с одной стороны, и энергичностью и громкостью с другой (-0,6 и -0,56 соответственно).Также достаточно высокие значения корреляции для танцевальности и громкости (0,51), а также для танцевальности и энергичности (0,44). Не очень высокой, но все же заметной корреляцией обладают акустичность и инструментальность (0,3). Для остальных пар характеристик корреляции не так высоки - ниже 0,3 по модулю.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

X=df[1:]
cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=3))])
cluster_pipeline.fit(X)
df['cluster'] = cluster_pipeline.predict(X)

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=2))])
df_embedding = tsne_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=df_embedding)
projection['energy'] = df['energy']
projection['loudness'] = df['loudness']
projection['cluster'] = df['cluster']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['energy','loudness'])
fig.show()

In [ ]:
for i in range (0,3):
    print("Cluster "+str(i)+": "+str(len(df[df['cluster']==i])))

In [ ]:
df_mean=df.groupby('cluster').mean()
df_mean.to_excel('means.xlsx')

In [ ]:
from scipy import stats
ttest_ind=stats.ttest_ind
d1 = df[df['cluster']==0].duration_ms
d2 = df[df['cluster']==2].duration_ms
print('Акустичность песен для кластера 1: ',d1.mean())
print('Акустичность песен для кластера 2: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
df[df['cluster']==1]['danceability']

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('energy ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
# output (ANOVA F and p value)

In [ ]:
model = ols('danceability ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
model = ols('acousticness ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
model = ols('duration_ms ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
model = ols('instrumentalness ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
model = ols('liveness ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
model = ols('loudness ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
model = ols('speechiness ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
model = ols('valence ~ C(cluster)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

## Сравнение с общей базой треков

In [ ]:
import pandas as pd

data = pd.read_csv('data.csv')
len(data)

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
df.describe()

C первого взгляда кажется, что для всех характеристик песен значения из песен в плейлисте сильно отличаются. Посмотрим конкретнее по каждой характеристике.

In [ ]:
from scipy import stats
ttest_ind=stats.ttest_ind
d1 = df.acousticness
d2 = data.acousticness
print('Акустичность песен из плейслистов: ',d1.mean())
print('Акустичность песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
d1 = df.danceability
d2 = data.danceability
print('Танцевальность песен из плейслистов: ',d1.mean())
print('Танцевальность песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
d1 = df.duration_ms
d2 = data.duration_ms
print('Длительность песен из плейслистов: ',d1.mean())
print('Длительность песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
d1 = df.energy
d2 = data.energy
print('Энергичность песен из плейслистов: ',d1.mean())
print('Энергичность песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
d1 = df.instrumentalness
d2 = data.instrumentalness
print('Инструментальность песен из плейслистов: ',d1.mean())
print('Инструментальность песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
d1 = df.liveness
d2 = data.liveness
print('Живое исполнение песен из плейслистов: ',d1.mean())
print('Живое исполнение песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
d1 = df.loudness
d2 = data.loudness
print('Громкость песен из плейслистов: ',d1.mean())
print('Громкость песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
d1 = df.speechiness
d2 = data.speechiness
print('Вокальность песен из плейслистов: ',d1.mean())
print('Вокальность песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

In [ ]:
d1 = df.valence
d2 = data.valence
print('Валентность песен из плейслистов: ',d1.mean())
print('Валентность песен из общей базы: ',d2.mean())
ttest_ind(d1,d2)

## Выводы

Была создана база данных с песнями, добавленными в плейлисты, созданные для обучения. Эту базу данных можно использовать для построения рекомендаций или в качестве основания для модели генерации музыки (если получится заполучить музыкальные отрывки для обучения модели).

Песни в плейлистах для учебы достаточно близки друг к другу по характеристикам (нет ярко выраженных кластеров). Было бы неплохо добавить переменную жанра, чтобы проверить, есть ли различные жанры в разных кластерах, которые были выделены. В целом по описанию плейлистов можно сделать вывод, что чаще всего для таких плейлистов выбираются классическая и инструментальная музыка.

Для песен из плейлистов для обучения наблюдается такие различия с общей базой:
* Акустичность выше
* Танцевальность ниже
* Длительность ниже
* Энергичность намного ниже
* Вероятность живого исполнения ниже
* Громкость ниже
* Вокальность ниже
* Валентность ниже

Эту информацию я планирую использовать в дипломе:

1) Для описания характеристик наиболее часто используемых в плейлистах песен (полученные результаты частично подтверждают данные из литературы)

2) Для подбора музыкальных отрывков для эксперимента, который планируется в рамках диплома (если будет достаточно времени)

In [ ]:
data1=pd.read_csv('data_by_genres.csv')
genres_energy=data1.sort_values(by=['energy']).head(50)

In [ ]:
genres_energy['genres']